# Coordinate Reference System Transformations

This notebook demonstrates CRS transformations and working with projected coordinate systems in Apache Sedona.

In [ ]:
from sedona.spark import SedonaContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from typing import Dict, Any
import math

In [ ]:
# Initialize Sedona with CRS support
def create_sedona_with_crs() -> SedonaContext:
    """Create Sedona session with CRS transformation support."""
    spark = SparkSession.builder \
        .appName("CRSTransformations") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryo.registrator", "org.apache.sedona.core.serde.SedonaKryoRegistrator") \
        .config("spark.sql.adaptive.enabled", "false") \
        .getOrCreate()
    
    return SedonaContext.create(spark)

sedona = create_sedona_with_crs()
print("Sedona session with CRS support initialized")

In [ ]:
# Sample GPS coordinates (WGS84 - EPSG:4326)
gps_coordinates_data = [
    ("Central Park", -73.9657, 40.7829),
    ("Golden Gate Bridge", -122.4783, 37.8199),
    ("Space Needle", -122.3493, 47.6205),
    ("Lincoln Memorial", -77.0502, 38.8893),
]

gps_schema = StructType([
    StructField("landmark", StringType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("latitude", DoubleType(), True)
])

gps_df = sedona.createDataFrame(gps_coordinates_data, gps_schema)
gps_df.createOrReplaceTempView("gps_points")

In [ ]:
# Transform WGS84 to Web Mercator (EPSG:3857) for web mapping
web_mercator_df = sedona.sql("""
    SELECT 
        landmark,
        longitude,
        latitude,
        ST_Point(longitude, latitude) as wgs84_geometry,
        ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857') as web_mercator_geometry,
        ST_X(ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857')) as mercator_x,
        ST_Y(ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857')) as mercator_y
    FROM gps_points
""")

print("WGS84 to Web Mercator transformation:")
web_mercator_df.show(truncate=False)

In [ ]:
# Transform to UTM zones for accurate distance measurements
utm_transform_df = sedona.sql("""
    SELECT 
        landmark,
        longitude,
        latitude,
        ST_Point(longitude, latitude) as original_point,
        CASE 
            WHEN longitude BETWEEN -126 AND -120 THEN ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:32610')
            WHEN longitude BETWEEN -120 AND -114 THEN ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:32611')
            WHEN longitude BETWEEN -78 AND -72 THEN ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:32618')
            ELSE ST_Point(longitude, latitude)
        END as utm_geometry,
        CASE 
            WHEN longitude BETWEEN -126 AND -120 THEN 'UTM Zone 10N (EPSG:32610)'
            WHEN longitude BETWEEN -120 AND -114 THEN 'UTM Zone 11N (EPSG:32611)'
            WHEN longitude BETWEEN -78 AND -72 THEN 'UTM Zone 18N (EPSG:32618)'
            ELSE 'No UTM transformation'
        END as utm_zone
    FROM gps_points
""")

print("UTM zone transformations by longitude:")
utm_transform_df.show(truncate=False)

In [ ]:
# Calculate accurate distances using projected coordinates
def calculate_projected_distances():
    """Calculate distances using appropriate projections."""
    distance_df = sedona.sql("""
        WITH projected_points AS (
            SELECT 
                landmark,
                longitude,
                latitude,
                ST_Point(longitude, latitude) as wgs84_point,
                ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857') as mercator_point
            FROM gps_points
        )
        SELECT 
            p1.landmark as landmark1,
            p2.landmark as landmark2,
            ST_Distance(p1.wgs84_point, p2.wgs84_point) as distance_degrees,
            ST_Distance(p1.mercator_point, p2.mercator_point) as distance_meters_approx,
            ST_Distance(p1.mercator_point, p2.mercator_point) / 1000 as distance_km_approx
        FROM projected_points p1
        CROSS JOIN projected_points p2
        WHERE p1.landmark != p2.landmark
        ORDER BY distance_degrees
    """)
    
    return distance_df

distance_comparison_df = calculate_projected_distances()
print("Distance calculations in different coordinate systems:")
distance_comparison_df.show()

In [ ]:
# Validate CRS transformations
validation_df = sedona.sql("""
    SELECT 
        landmark,
        ST_IsValid(ST_Point(longitude, latitude)) as wgs84_valid,
        ST_IsValid(ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857')) as mercator_valid,
        ST_SRID(ST_Point(longitude, latitude)) as original_srid,
        ST_SRID(ST_Transform(ST_Point(longitude, latitude), 'EPSG:4326', 'EPSG:3857')) as transformed_srid
    FROM gps_points
""")

print("CRS transformation validation:")
validation_df.show()

## CRS Transformation Guidelines

1. **WGS84 (EPSG:4326)**: Standard GPS coordinates, degrees
2. **Web Mercator (EPSG:3857)**: Web mapping, approximate distances
3. **UTM**: Accurate measurements, choose appropriate zone
4. **Always validate**: Use `ST_IsValid()` after transformations
5. **Performance**: Cache transformed geometries for repeated use

### Common EPSG Codes
- 4326: WGS84 (GPS coordinates)
- 3857: Web Mercator (web maps)  
- 32610-32660: UTM Northern Hemisphere zones
- 32710-32760: UTM Southern Hemisphere zones